# Monitor

[![Github](https://img.shields.io/github/stars/lab-ml/labml?style=social)](https://github.com/lab-ml/labml)
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lab-ml/labml/blob/master/guides/monitor.ipynb)
[![Docs](https://img.shields.io/badge/labml-docs-blue)](https://docs.labml.ai/api/monit.html)

In [ ]:
%%capture
!pip install labml

## Iterators & Enumerators

You can use [labml.monit.iterate](https://docs.labml.ai/api/monit.html#labml.monit.iterate) and [labml.monit.enum](https://docs.labml.ai/api/monit.html#labml.monit.enum) with any iterable object.

In this example we use a PyTorch `DataLoader`.

In [1]:
# Create a data loader for illustration
import time

import torch
from torchvision import datasets, transforms

from labml import logger, monit, lab, tracker

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST(lab.get_data_path(),
                       train=False,
                       download=True,
                       transform=transforms.Compose([
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=32, shuffle=True)

In [2]:
for data, target in monit.iterate("Test", test_loader):
    time.sleep(0.01)

In [3]:
for i, (data, target) in monit.enum("Test", test_loader):
    time.sleep(0.01)

## Sections

[labml.monit.section](https://docs.labml.ai/api/monit.html#labml.monit.section) lets you monitor time taken for different tasks and also helps keep the code clean by separating different blocks of code.

In [4]:
with monit.section("Load data"):
    # code to load data
    time.sleep(2)

In [5]:
with monit.section("Load saved model"):
    time.sleep(1)
    monit.fail()

You can also show progress while a section is running

In [6]:
with monit.section("Train"):
    for i in range(100):
        time.sleep(0.1)
        # Multiple training steps in the inner loop
        monit.progress(i / 100)

You can use [labml.monit.func](https://docs.labml.ai/api/monit.html#labml.monit.func) as a decorator.

In [7]:
@monit.func('Train')
def train():
    for i in range(100):
        time.sleep(0.1)
        # Multiple training steps in the inner loop
        monit.progress(i / 100)    

train()

## Mix

[labml.monit.mix](https://docs.labml.ai/api/monit.html#labml.monit.mix) mixes mulutiple iterators. This is useful when you want to switch between training and validation within a training epoch.

The following loop mixes two iterators `2` times. As in it changes between the two iterators only twice.

In [8]:
for t, idx in monit.mix(2, ('train', range(0, 32, 8)), ('valid', 4)):
    logger.log(f'{t}; {idx}')

You can ignore the first parameter and it'll mix as often as possible.

In [9]:
for t, idx in monit.mix(('train', range(0, 32, 8)), ('valid', 4)):
    logger.log(f'{t}; {idx}')

You can specify functions to be called instead of iterating also.

In [10]:
def sample(idx):
    logger.log(f'sample: {idx}')
    
for t, idx in monit.mix(('train', range(0, 32, 8)), ('valid', 4), (sample, 2)):
    logger.log(f'{t}; {idx}')

## Loop

This can be used for the training loop. The [labml.monit.loop](https://docs.labml.ai/api/monit.html#labml.monit.loop) keeps track of the time taken and time remaining for the loop.

[labml.tracker.save](https://docs.labml.ai/api/tracker.html#labml.tracker.save) outputs the current status along with global step.

In [11]:
for step in monit.loop(range(0, 400)):
    tracker.save()

You can manually increment global step too.

In [12]:
tracker.set_global_step(0) # Reset the counter because we incremented in previous cell

for step in monit.loop(range(0, 400)):
    tracker.add_global_step(5)
    tracker.save()

You can use [labml.monit.mix](https://docs.labml.ai/api/monit.html#labml.monit.mix) within a `loop`.

In [15]:
tracker.set_global_step(0) # Reset the counter because we incremented in previous cell

for _ in monit.loop(2):
    for t, idx in monit.mix(('train', range(50)), ('valid', range(10))):
        time.sleep(0.05)
        tracker.save({t: idx})
        if t == 'train':
            tracker.add_global_step()
    tracker.new_line()
